In [ ]:
!pip install -qU langchain_google_genai langgraph fastapi uvicorn nest-asyncio pyngrok streamlit requests


In [ ]:
from getpass import getpass
import os
from pyngrok import ngrok

os.environ["GOOGLE_API_KEY"] = getpass("enter google api key")


ngrok_authtoken = getpass("Enter your ngrok authtoken: ")
if ngrok_authtoken:
    ngrok.set_auth_token(ngrok_authtoken)
    print("ngrok authtoken set.")
else:
    print("No ngrok authtoken provided — you must provide it to expose services publicly from Colab.")


enter google api key··········
Enter your ngrok authtoken: ··········
ngrok authtoken set.


In [ ]:
backend_code = r'''
import os
import json
import datetime
from fastapi import FastAPI
from pydantic import BaseModel
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict
from typing import Annotated
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0.7,
)

class ChatState(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder_chat = StateGraph(ChatState)

def chatbot(state: ChatState) -> ChatState:
    return {"messages": gemini_llm.invoke(state["messages"])}

graph_builder_chat.add_node("chatbot", chatbot)
graph_builder_chat.add_edge(START, "chatbot")
graph_builder_chat.add_edge("chatbot", END)

memory = MemorySaver()
graph_memory = graph_builder_chat.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "1"}}

app = FastAPI()

class UserMessage(BaseModel):
    message: str

HISTORY_FILE = "chat_history.json"

def extract_text(msg):
    if isinstance(msg, dict):
        return msg.get("content") or msg.get("text") or msg.get("message") or str(msg)
    if hasattr(msg, "content"):
        return getattr(msg, "content")
    return str(msg)

@app.post("/chat")
async def chat(user_message: UserMessage):
    response = graph_memory.invoke(
        ChatState(messages=[{"role": "user", "content": user_message.message}]),
        config
    )
    last = response["messages"][-1]
    reply = extract_text(last)
    try:
        with open(HISTORY_FILE, "r") as f:
            history = json.load(f)
    except FileNotFoundError:
        history = []
    history.append({
        "user": user_message.message,
        "bot": reply,
        "timestamp": datetime.datetime.utcnow().isoformat() + "Z"
    })
    with open(HISTORY_FILE, "w") as f:
        json.dump(history, f, indent=2)
    return {"reply": reply}
'''

with open("backend.py", "w") as f:
    f.write(backend_code)

print("backend.py written to disk.")


backend.py written to disk.


In [ ]:
import time
from pyngrok import ngrok

get_ipython().system_raw("uvicorn backend:app --host 0.0.0.0 --port 8000 &")

try:
    public_tunnel = ngrok.connect(8000)
    backend_public_url = public_tunnel.public_url
    print("✅ FastAPI public URL:", backend_public_url)
    print("👉 Open this link in a new browser tab to check the backend.")
except Exception as e:
    backend_public_url = "http://127.0.0.1:8000"
    print("⚠️ Couldn't open ngrok tunnel (maybe authtoken missing). Using local URL:", backend_public_url)


✅ FastAPI public URL: https://unreferred-jo-uredinial.ngrok-free.dev
👉 Open this link in a new browser tab to check the backend.


In [ ]:
backend_url = globals().get("backend_public_url", "http://127.0.0.1:8000")
streamlit_code = f'''
import streamlit as st
import requests
import json
import os

BACKEND_URL = "{backend_url}"

st.set_page_config(page_title="Gemini Chatbot", page_icon="🤖")
st.title("🤖 Gemini Chatbot (LangGraph + FastAPI)")

if "messages" not in st.session_state:
    st.session_state.messages = []

def send_message(text):
    resp = requests.post(f"{{BACKEND_URL}}/chat", json={{"message": text}}, timeout=120)
    return resp.json().get("reply")

with st.form("chat_form", clear_on_submit=True):
    user_input = st.text_input("You:")
    submitted = st.form_submit_button("Send")
    if submitted and user_input:
        reply = send_message(user_input)
        st.session_state.messages.append({{"sender":"You", "text": user_input}})
        st.session_state.messages.append({{"sender":"Bot", "text": reply}})

for msg in st.session_state.messages:
    if msg["sender"] == "You":
        st.markdown(f"**You:** {{msg['text']}}")
    else:
        st.markdown(f"**Bot:** {{msg['text']}}")

if st.button("Save chat to file (downloadable)"):
    with open("chat_history_saved.json", "w") as f:
        json.dump(st.session_state.messages, f, indent=2)
    st.success("Saved to chat_history_saved.json (see files on the left).")
'''

with open("app.py", "w") as f:
    f.write(streamlit_code)

print("app.py written (Streamlit frontend). Backend URL used:", backend_url)


app.py written (Streamlit frontend). Backend URL used: https://unreferred-jo-uredinial.ngrok-free.dev


In [ ]:
from pyngrok import ngrok
from IPython.display import display, Markdown

# Start Streamlit server in background
get_ipython().system_raw("streamlit run app.py --server.port 8501 &")

try:
    streamlit_url = ngrok.connect(8501).public_url
    print("Streamlit public URL:", streamlit_url)
except Exception as e:
    streamlit_url = "http://127.0.0.1:8501"
    print("Couldn't open ngrok tunnel for Streamlit (maybe authtoken missing). Streamlit local URL:", streamlit_url)

# Display clickable link in Colab output
display(Markdown(f"[Open Streamlit app]({streamlit_url})"))


Streamlit public URL: https://unreferred-jo-uredinial.ngrok-free.dev


[Open Streamlit app](https://unreferred-jo-uredinial.ngrok-free.dev)